# Collaborative Filtering

1. Simple Collaborative Filtering
- User-based CF
- Item-based CF
- Matrix Factorization

2. Advanced Collaborative Filtering
- Neural Collaborative Filtering
- Deep Matrix Factorization



In [2]:
# reset directory
%reset -f

# load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# load data from csv file
# amazon = pd.read_csv('/Users/pavansingh/Library/CloudStorage/GoogleDrive-pavansingho23@gmail.com/My Drive/Portfolio/Masters-Dissertation/Code/Data/amz_with_senti_1.csv')

# load sample data from csv file
amazon = pd.read_csv('/Users/pavansingh/Library/CloudStorage/GoogleDrive-pavansingho23@gmail.com/My Drive/Portfolio/Masters-Dissertation/Code/Data/amz_with_senti_sample_1.csv')

# check data 
print("Shape of data", amazon.shape)
print("Number of unique users", amazon.reviewerID.nunique())
print("Number of unique products", amazon.asin.nunique())
display(amazon.head(4))

Shape of data (403550, 17)
Number of unique users 161210
Number of unique products 227151


,reviewerID,reviewerName,reviewTime,asin,reviewText,category,overall,normalized_rating,stemmed_words_revText,lemmatized_words_revText,filtered_tokens_revText,sentiments_vader_revText,sentiments_textblob_revText,subjectivities_textblob_revText,sentiment_score_afinn_revText,sentiment_score_bing_revText,sentiment_score_nrc_revText
0,A29NAG6NZOBAJ8,kingpin16,2014-11-24,B001IH8ERA,tuna yum,grocery_and_gourmet_food,5.0,1.00,"['tuna', 'yum']","['tuna', 'yum']","['tuna', 'yum']","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000000,0.000000,0,0,trust
1,A1WVA7V02PQOY6,Dad of Divas,2015-02-10,B000ZGY4PG,as someone that has always liked eating oatmea...,grocery_and_gourmet_food,5.0,1.00,"['someon', 'alway', 'like', 'eat', 'oatmeal', ...","['someone', 'always', 'liked', 'eating', 'oatm...","['someone', 'always', 'liked', 'eating', 'oatm...","{'neg': 0.0, 'neu': 0.82, 'pos': 0.18, 'compou...",0.397564,0.705641,4,4,positive
2,A1KQJLBDF2OEMD,Sherelle Ellis,2015-07-28,B00YLLHNHW,humans are stupid they love and they make mist...,kindle_store,4.0,0.75,"['human', 'stupid', 'love', 'make', 'mistak', ...","['human', 'stupid', 'love', 'make', 'mistake',...","['humans', 'stupid', 'love', 'make', 'mistakes...","{'neg': 0.109, 'neu': 0.703, 'pos': 0.188, 'co...",0.097186,0.688095,4,1,sadness
3,A1MUHTKSOY7WVO,Upgrade Taos Computers,2015-05-20,B00GJU4DD0,this thing rocks very lightslim great fit does...,electronics,5.0,1.00,"['thing', 'rock', 'lightslim', 'great', 'fit',...","['thing', 'rock', 'lightslim', 'great', 'fit',...","['thing', 'rocks', 'lightslim', 'great', 'fit'...","{'neg': 0.0, 'neu': 0.512, 'pos': 0.488, 'comp...",0.466667,0.483333,3,1,trust


#  Simple Collaborative Filtering

## User-Based Collaborative Filtering

1. change data to user-item matrix
2. calculate similarity between users
3. predict rating for each user-item pair
4. recommend items with highest predicted ratings